In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__)

In [ ]:
tf.enable_eager_execution()

In [ ]:
#DATA_FILE = './diabetes.csv'
DATA_FILE = '/content/gdrive/My Drive/TensorFlow_Training_12th/data/diabetes.csv'

In [ ]:
xy = np.loadtxt(DATA_FILE, delimiter=',', dtype=np.float32)
x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

print(x_train.shape, y_train.shape)
print(xy)

In [ ]:
batch_size = x_train.shape[0]
n_epoch = 1000
learning_rate = 0.1

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(batch_size)

In [ ]:
w = tf.Variable(tf.random_normal([8, 1]))
b = tf.Variable(tf.random_normal([1]))

In [ ]:
def logistic_regression(inputs):
    hypothesis = tf.keras.activations.sigmoid(tf.matmul(inputs, w) + b)
    return hypothesis

In [ ]:
def loss_fn(inputs, labels):
    hypothesis = logistic_regression(inputs)
    loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(labels, hypothesis))
    return loss

In [ ]:
def grad(inputs, labels):
    hypothesis = logistic_regression(inputs)
    with tf.GradientTape() as tape:
        loss = loss_fn(inputs, labels)
    grads = tape.gradient(loss, [w, b])
    return grads

In [ ]:
def accuracy_fn(inputs, labels):
    hypothesis = logistic_regression(inputs)
    prediction = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, labels), dtype=tf.float32))
    return accuracy

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

In [ ]:
total_steps = int(x_train.shape[0]/batch_size)
for epoch in range(n_epoch):
    total_loss = 0.
    for x, y in dataset: 
        grads = grad(x, y)        
        optimizer.apply_gradients(grads_and_vars=zip(grads, [w, b]))
        loss = loss_fn(x, y)
        total_loss += loss / total_steps
    if (epoch+1) % 10 == 0:
        print('Epoch {0}: {1:.8f}'.format(epoch+1, total_loss))

In [ ]:
print('Accuracy: {}'.format(accuracy_fn(x_train, y_train)))